In [1]:
pip install requests beautifulsoup4 pandas


You should consider upgrading via the '/Users/Nicolas/.pyenv/versions/3.8.10/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


In [3]:
url = "https://www.leboncoin.fr/recherche?category=9&locations=Créteil_94000__48.77745_2.45518_3377&real_estate_type=2"

In [4]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


In [5]:
# Fonction pour nettoyer et récupérer les chiffres
def get_numbers(text):
    return float(re.sub(r'[^\d,]', '', text).replace(',', '.'))


In [6]:
# Liste pour collecter les annonces
annonces = []


In [7]:
# Extraction simple des annonces sur la page
for annonce in soup.select('div[data-qa-id="aditem_container"]'):
    title = annonce.select_one('p[data-qa-id="aditem_title"]').text.strip()
    price_tag = annonce.select_one('span[data-qa-id="aditem_price"]')
    surface_tag = annonce.select_one('div[data-qa-id="aditem_description"]')

In [8]:
 if price_tag and surface_tag:
        price = get_numbers(price_tag.text)
        

NameError: name 'price_tag' is not defined

In [9]:
for annonce in soup.select('div[data-qa-id="aditem_container"]'):
    title = annonce.select_one('p[data-qa-id="aditem_title"]').text.strip()
    price_tag = annonce.select_one('span[data-qa-id="aditem_price"]')
    surface_tag = annonce.select_one('div[data-qa-id="aditem_description"]')

    if price_tag and surface_tag:
        price = get_numbers(price_tag.text)

        surface_match = re.search(r'(\d+)\s?m²', surface_tag.text)
        if surface_match:
            surface = float(surface_match.group(1))
            prix_m2 = price / surface

            prix_revente_simule = prix_m2 * surface * 1.2
            rentabilite = prix_revente_simule - price

            annonces.append({
                'Titre': title,
                'Prix (€)': price,
                'Surface (m²)': surface,
                'Prix au m² (€)': round(prix_m2, 2),
                'Prix Revente Simulé (+20%) (€)': round(prix_revente_simule, 2),
                'Rentabilité simulée (€)': round(rentabilite, 2),
            })

df = pd.DataFrame(annonces)
print(df)


Empty DataFrame
Columns: []
Index: []


In [10]:
pip install selenium webdriver-manager


You should consider upgrading via the '/Users/Nicolas/.pyenv/versions/3.8.10/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import re
import time

url = "https://www.leboncoin.fr/recherche?category=9&locations=Créteil_94000__48.77745_2.45518_3377&real_estate_type=2"

# Préparer le navigateur Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)
time.sleep(5)  # Temps pour charger les annonces

# Fonction pour nettoyer et récupérer les chiffres
def get_numbers(text):
    return float(re.sub(r'[^\d,]', '', text).replace(',', '.'))

annonces = []

# Extraction annonces avec Selenium
cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-qa-id="aditem_container"]')

for annonce in cards:
    try:
        title = annonce.find_element(By.CSS_SELECTOR, 'p[data-qa-id="aditem_title"]').text.strip()
        price_text = annonce.find_element(By.CSS_SELECTOR, 'span[data-qa-id="aditem_price"]').text
        description = annonce.find_element(By.CSS_SELECTOR, 'div[data-qa-id="aditem_description"]').text

        price = get_numbers(price_text)
        
        surface_match = re.search(r'(\d+)\s?m²', description)
        if surface_match:
            surface = float(surface_match.group(1))
            prix_m2 = price / surface

            prix_revente_simule = prix_m2 * surface * 1.2
            rentabilite = prix_revente_simule - price

            annonces.append({
                'Titre': title,
                'Prix (€)': price,
                'Surface (m²)': surface,
                'Prix au m² (€)': round(prix_m2, 2),
                'Prix Revente Simulé (+20%) (€)': round(prix_revente_simule, 2),
                'Rentabilité simulée (€)': round(rentabilite, 2),
            })
    except Exception as e:
        print("Annonce ignorée (info manquante)")

driver.quit()

df = pd.DataFrame(annonces)
print(df)

df.to_csv('annonces_immo_test.csv', index=False)


Empty DataFrame
Columns: []
Index: []


In [12]:
import re
import pandas as pd

# Charger ton fichier texte copié depuis LeBonCoin
with open('annonces.txt', 'r', encoding='utf-8') as file:
    contenu = file.read()

# Trouver toutes les annonces (prix et surface)
prix = re.findall(r'(\d{2,3}\s?\d{0,3})\s?€', contenu)
surfaces = re.findall(r'(\d{1,3})\s?m²', contenu)

annonces = []

# On limite aux annonces pour lesquelles on a à la fois prix et surface
for p, s in zip(prix, surfaces):
    price = float(p.replace(' ', '').replace(',', '.'))
    surface = float(s)
    prix_m2 = price / surface
    prix_revente_simule = prix_m2 * surface * 1.2
    rentabilite = prix_revente_simule - price

    annonces.append({
        'Prix (€)': price,
        'Surface (m²)': surface,
        'Prix au m² (€)': round(prix_m2, 2),
        'Prix Revente Simulé (+20%) (€)': round(prix_revente_simule, 2),
        'Rentabilité simulée (€)': round(rentabilite, 2),
    })

df = pd.DataFrame(annonces)
print(df)

# Export en CSV
df.to_csv('annonces_immo_test.csv', index=False)


    Prix (€)  Surface (m²)  Prix au m² (€)  Prix Revente Simulé (+20%) (€)  \
0      550.0          50.0           11.00                           660.0   
1      280.0          47.0            5.96                           336.0   
2      150.0          44.0            3.41                           180.0   
3      700.0          60.0           11.67                           840.0   
4      990.0          32.0           30.94                          1188.0   
5      690.0          16.0           43.12                           828.0   
6      490.0          79.0            6.20                           588.0   
7      970.0          33.0           29.39                          1164.0   
8      910.0          30.0           30.33                          1092.0   
9      200.0          28.0            7.14                           240.0   
10     902.0          38.0           23.74                          1082.4   
11     386.0          61.0            6.33                      

In [ ]:
import re
import pandas as pd

# Charger le contenu du fichier texte annonces.txt
with open('annonces.txt', 'r', encoding='utf-8') as file:
    contenu = file.read()

# Extraction automatique des prix et surfaces
prix = re.findall(r'(\d{2,3}\s?\d{0,3})\s?€', contenu)
surfaces = re.findall(r'(\d{1,3})\s?m²', contenu)

annonces = []

# Calcul automatique de rentabilité pour chaque annonce
for p, s in zip(prix, surfaces):
    price = float(p.replace(' ', '').replace(',', '.'))
    surface = float(s)
    prix_m2 = price / surface
    prix_revente_simule = prix_m2 * surface * 1.2
    rentabilite = prix_revente_simule - price

    annonces.append({
        'Prix (€)': price,
        'Surface (m²)': surface,
        'Prix au m² (€)': round(prix_m2, 2),
        'Prix Revente Simulé (+20%) (€)': round(prix_revente_simule, 2),
        'Rentabilité simulée (€)': round(rentabilite, 2),
    })

# Créer le tableau (DataFrame)
df = pd.DataFrame(annonces)

# Afficher le tableau
print(df)

# Exporter le résultat en CSV
df.to_csv('annonces_immo_test.csv', index=False)


In [13]:
pip install requests beautifulsoup4 pandas


You should consider upgrading via the '/Users/Nicolas/.pyenv/versions/3.8.10/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

url = "https://www.pap.fr/annonce/vente-appartements-creteil-94000-g43319-2-pieces"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

def get_numbers(text):
    return float(re.sub(r'[^\d,]', '', text).replace(',', '.'))

annonces = []

# PAP : Extraction des annonces
for annonce in soup.select('.search-list-item'):
    title = annonce.select_one('.item-title').get_text(strip=True)
    price_tag = annonce.select_one('.item-price')
    details = annonce.select_one('.item-tags').get_text(strip=True)

    if price_tag and details:
        price = get_numbers(price_tag.get_text())
        surface_match = re.search(r'(\d+)\s?m²', details)
        if surface_match:
            surface = float(surface_match.group(1))
            prix_m2 = price / surface
            prix_revente_simule = prix_m2 * surface * 1.2
            rentabilite = prix_revente_simule - price

            annonces.append({
                'Titre': title,
                'Prix (€)': price,
                'Surface (m²)': surface,
                'Prix au m² (€)': round(prix_m2, 2),
                'Prix Revente Simulé (+20%) (€)': round(prix_revente_simule, 2),
                'Rentabilité simulée (€)': round(rentabilite, 2),
            })

df = pd.DataFrame(annonces)
print(df)

# Export rapide CSV
df.to_csv('annonces_pap.csv', index=False)


Empty DataFrame
Columns: []
Index: []


In [17]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

url = "https://www.logic-immo.com/classified-search?distributionTypes=Buy,Buy_Auction,Compulsory_Auction&estateTypes=House,Apartment&locations=AD08FR36687&order=Default&m=homepage_new_search_classified_search_result"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

annonces = []

def get_numbers(text):
    return float(re.sub(r'[^\d,]', '', text).replace(',', '.'))

cards = soup.select('.offer-link')

print(f"Annonces trouvées : {len(cards)}")

for card in cards:
    try:
        title_tag = card.select_one('.offer-type')
        price_tag = card.select_one('.offer-price')
        details_tag = card.select_one('.offer-details')

        if title_tag and price_tag and details_tag:
            title = title_tag.get_text(strip=True)
            price = float(re.sub(r'\D', '', price_tag.get_text()))
            
            surface_match = re.search(r'(\d+)\s?m²', details_tag.get_text(strip=True))
            if surface_match:
                surface = float(surface_match.group(1))
                prix_m2 = price / surface
                prix_revente_simule = prix_m2 * surface * 1.2
                rentabilite = prix_revente_simule - price

                annonces.append({
                    'Titre': title,
                    'Prix (€)': price,
                    'Surface (m²)': surface,
                    'Prix au m² (€)': round(prix_m2, 2),
                    'Prix Revente Simulé (+20%) (€)': round(prix_revente_simule, 2),
                    'Rentabilité simulée (€)': round(rentabilite, 2),
                })

df = pd.DataFrame(annonces)

print(df)

df.to_csv('annonces_logic_immo.csv', index=False)


IndentationError: unexpected unindent (1030472990.py, line 50)